In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
import os
os.chdir('/home/jaentrouble/adipose_train2')
print(os.getcwd())

/home/jaentrouble/adipose_train2


In [3]:
!git pull
%autoreload

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 2.29 KiB | 1.14 MiB/s, done.
From https://github.com/jaentrouble/Adipos_tf_train2
   8c1fffd..41cb1af  master     -> origin/master
Updating 8c1fffd..41cb1af
Fast-forward
 model_trainer.py  | 10 +++++++-
 view_result.ipynb | 75 ++++++++++++++++++++++++++++++++++++++++++++-----------
 2 files changed, 70 insertions(+), 15 deletions(-)


In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
with np.load('cell_mask_data.npz') as data:
    X = data['img']
    Y = data['mask']
print('X shape: ', X.shape, X.dtype)
print('Y shape: ', Y.shape, Y.dtype)

X shape:  (1533, 200, 200, 3) uint8
Y shape:  (1533, 200, 200) float32


In [6]:
X_test = X[1350:]
Y_test = Y[1350:]

In [7]:
import tensorflow as tf
from model_trainer import get_model
from adipose_models import *

In [8]:
model_f = conv4_b2_0
savedfile_name = 'conv4_b2_0'
target_epoch = 10

In [9]:
test_model = get_model(conv4_b2_0)
test_model.load_weights('savedmodels/'+savedfile_name+'/'+str(target_epoch))

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200, 32)     

NameError: name 'loss' is not defined

In [12]:
test_model.evaluate(X_test, Y_test)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.